**//IMPORTS**

In [2]:
import duckdb
import pandas as pd, pathlib, itertools, textwrap

**//CONFIGS**

In [3]:
DB_PATH = '../data/duckdb/subs.duckdb'
pd.set_option("display.max_colwidth", None)


**//MAIN CODE**

In [4]:
con = duckdb.connect(pathlib.Path('../data/duckdb/subs.duckdb'), read_only=True)
con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='main'").fetchall()



IOException: IO Error: Could not set lock on file "/home/laiarodrigo/repos/Thesis/eda's/../data/duckdb/subs.duckdb": Conflicting lock is held in /usr/bin/python3.12 (PID 387580). See also https://duckdb.org/docs/stable/connect/concurrency

**//LOAD TEST DATASETS**

In [14]:
GITHUB_RAW = ("https://raw.githubusercontent.com/google-research/google-research/"
              "HEAD/frmt/dataset")

def read_file(bucket, region):
    url = f"{GITHUB_RAW}/{bucket}_bucket/pt_{bucket}_test_en_pt-{region}.tsv"
    return pd.read_csv(url, sep="\t", names=["en", "pt"])

def report_dupes(df, tag):
    dupes = df[df["en"].duplicated(keep=False)]
    if not dupes.empty:
        print(f"\n⚠️  {tag}: {dupes['en'].nunique()} distinct English "
              f"sentences duplicated ({len(dupes)} rows).")
        # show the first few duplicates
        print(textwrap.indent(dupes.head(6).to_string(index=False), "    "))
    return dupes

buckets = {          # bucket → filename prefix inside that bucket
    "lexical": "pt_lexical",
    "entity" : "pt_entity",
    "random" : "pt_random",
}

splits   = ["dev", "test", "exemplars"]        # the paper’s three splits
regions  = ["pt-BR", "pt-PT"]                  # ⬅️ we ignore zh-* files

def urls(bucket):
    prefix = buckets[bucket]
    return [f"{GITHUB_RAW}/{bucket}_bucket/"
            f"{prefix}_{split}_en_{region}.tsv"
            for split in splits
            for region in regions]

# sanity-check
print(urls("entity")[:3])

def split_key(bucket, split, region):
    # pt-BR  ➜  pt_BR   (dash → underscore)
    return f"{bucket}_{split}_{region.replace('-', '_')}"

data_files = {
    split_key(bucket, split, region): [
        f"{GITHUB_RAW}/{bucket}_bucket/"
        f"{buckets[bucket]}_{split}_en_{region}.tsv"
    ]
    for bucket in buckets
    for split  in splits
    for region in regions
}

from datasets import load_dataset, DatasetDict

ds = load_dataset(
        "csv",
        data_files   = data_files,
        delimiter    = "\t",
        column_names = ["en", "pt"],
)

print(list(ds.keys())[:6])
# ['lexical_dev_pt_BR', 'lexical_dev_pt_PT', 'lexical_test_pt_BR', …]

all_frames = []

for bucket, region in itertools.product(["lexical", "entity", "random"], ["BR","PT"]):
    df = read_file(bucket, region)
    report_dupes(df, f"{bucket}-{region}")
    # keep first occurrence only
    df = df.drop_duplicates(subset="en", keep="first")
    all_frames.append(((bucket, region), df))

['https://raw.githubusercontent.com/google-research/google-research/HEAD/frmt/dataset/entity_bucket/pt_entity_dev_en_pt-BR.tsv', 'https://raw.githubusercontent.com/google-research/google-research/HEAD/frmt/dataset/entity_bucket/pt_entity_dev_en_pt-PT.tsv', 'https://raw.githubusercontent.com/google-research/google-research/HEAD/frmt/dataset/entity_bucket/pt_entity_test_en_pt-BR.tsv']
['lexical_dev_pt_BR', 'lexical_dev_pt_PT', 'lexical_test_pt_BR', 'lexical_test_pt_PT', 'lexical_exemplars_pt_BR', 'lexical_exemplars_pt_PT']

⚠️  lexical-BR: 4 distinct English sentences duplicated (8 rows).
                                                                                                                                                                                                                    en                                                                                                                                                                                                

In [ ]:
# con = duckdb.connect(DB_PATH)

# # --------------------------------------------------
# # 1)  create sequence + test_data table (if absent)
# # --------------------------------------------------
# con.execute("""
# CREATE SEQUENCE IF NOT EXISTS seq_test_pairs START 1;
# CREATE TABLE IF NOT EXISTS test_data (
#     test_id     BIGINT PRIMARY KEY DEFAULT nextval('seq_test_pairs'),
#     dataset     TEXT NOT NULL,          -- 'FRMT' / 'Gold' / …
#     bucket      TEXT NOT NULL,          -- 'lexical' / 'entity' / 'random'
#     text_pt_br  TEXT,
#     text_pt_pt  TEXT
# );
# """)

# aligned_frames = []

# for bucket in ["lexical", "entity", "random"]:
#     br = dict(all_frames)[(bucket, "BR")]
#     pt = dict(all_frames)[(bucket, "PT")]

#     merged = br.merge(pt, on="en", validate="one_to_one")  # now guaranteed
#     aligned_frames.append(
#         merged.assign(dataset="FRMT", bucket=bucket)
#               .rename(columns={"pt_x": "text_pt_br",
#                                "pt_y": "text_pt_pt"})
#               [["dataset", "bucket", "text_pt_br", "text_pt_pt"]]
#     )

# frmt_test = pd.concat(aligned_frames, ignore_index=True)
# print("Rows after merge:", len(frmt_test))    # ≈ 2 616 if nothing was lost


Rows after merge: 2608


In [ ]:
# with duckdb.connect("../data/duckdb/subs.duckdb") as con:
#     con.register("tmp_frmt", frmt_test)

#     con.execute("""
#         INSERT INTO test_data (dataset, bucket, text_pt_br, text_pt_pt)
#         SELECT      dataset, bucket, text_pt_br, text_pt_pt
#         FROM tmp_frmt
#     """)

#     con.unregister("tmp_frmt")


In [4]:
con.close()


NameError: name 'con' is not defined

In [ ]:
# gold_ds   = load_dataset("joaosanches/golden_collection")

# df_gold = pd.DataFrame({
#     "dataset"        : "Gold Collection",
#     "bucket"         : "n/a",                 # Gold set isn’t bucketed
#     "text_pt_br"     : gold_ds["gold_collection"]["text"],   # bp source
#     "text_pt_pt"     : gold_ds["referencia_manual"]["text"], # human ref
#     "text_pt_pt_DeepL" : gold_ds["referencia_DeepL"]["text"]   # DeepL ref
# })

# with duckdb.connect("../data/duckdb/subs.duckdb") as con:
#     con.register("tmp_gold", df_gold)
#     con.execute("""
#         INSERT INTO test_data
#               (dataset, bucket, text_pt_br, text_pt_pt, text_pt_pt_DeepL)
#         SELECT dataset, bucket, text_pt_br, text_pt_pt, text_pt_pt_DeepL
#         FROM   tmp_gold;
#     """)
#     con.unregister("tmp_gold")
#     con.close()

In [28]:
con = duckdb.connect(DB_PATH)
df = con.execute("SELECT * FROM test_data LIMIT 5").fetchdf()
con.close()
df

,test_id,dataset,bucket,text_pt_br,text_pt_pt,text_pt_pt_DeepL
0,1,FRMT,lexical,"Um ônibus (contração de ômnibus, com as variações multibus, motorbus, autobus etc.) é um veículo rodoviário projetado para transportar muitos passageiros.",Um autocarro é um veículo rodoviário concebido para transportar muitos passageiros.,None
1,2,FRMT,lexical,Ônibus podem ter capacidade para até 300 passageiros.,Os autocarros têm capacidade para transportar no máximo 300 passageiros.,None
2,3,FRMT,lexical,"O tipo de ônibus mais comum é o ônibus rígido de um andar, com cargas maiores transportadas por ônibus de dois andares e articulados, e cargas menores transportadas por ônibus médios e micro-ônibus, enquanto ônibus executivos são usados para serviços de distâncias maiores.","Os tipos de autocarros mais comuns são os rígidos de um andar, sendo as cargas maiores nos de dois andares e os autocarros articulados. As cargas menores transportadas por autocarros médios e miniautocarros e enquanto as camionetas de passageiros são utilizadas para serviços de maior distância.",None
3,4,FRMT,lexical,"Muitos tipos de ônibus, como os de transporte urbano e os intermunicipais, cobram uma tarifa.","Muitos tipos de autocarros, como os de trânsito urbano e intercidades, cobram bilhete.",None
4,5,FRMT,lexical,"Outros tipos, como ônibus escolares do ensino médio ou fundamental ou ônibus de traslado em uma instituição de ensino superior não cobram tarifas.","Outros tipos, como os autocarros da escola primária ou secundária ou de ida e volta num campus pós-secundário não cobram bilhete.",None


In [29]:
con = duckdb.connect(DB_PATH)
df = con.execute("SELECT * FROM test_data").fetchdf()
con.close()
df

,test_id,dataset,bucket,text_pt_br,text_pt_pt,text_pt_pt_DeepL
0,1,FRMT,lexical,"Um ônibus (contração de ômnibus, com as variações multibus, motorbus, autobus etc.) é um veículo rodoviário projetado para transportar muitos passageiros.",Um autocarro é um veículo rodoviário concebido para transportar muitos passageiros.,None
1,2,FRMT,lexical,Ônibus podem ter capacidade para até 300 passageiros.,Os autocarros têm capacidade para transportar no máximo 300 passageiros.,None
2,3,FRMT,lexical,"O tipo de ônibus mais comum é o ônibus rígido de um andar, com cargas maiores transportadas por ônibus de dois andares e articulados, e cargas menores transportadas por ônibus médios e micro-ônibus, enquanto ônibus executivos são usados para serviços de distâncias maiores.","Os tipos de autocarros mais comuns são os rígidos de um andar, sendo as cargas maiores nos de dois andares e os autocarros articulados. As cargas menores transportadas por autocarros médios e miniautocarros e enquanto as camionetas de passageiros são utilizadas para serviços de maior distância.",None
3,4,FRMT,lexical,"Muitos tipos de ônibus, como os de transporte urbano e os intermunicipais, cobram uma tarifa.","Muitos tipos de autocarros, como os de trânsito urbano e intercidades, cobram bilhete.",None
4,5,FRMT,lexical,"Outros tipos, como ônibus escolares do ensino médio ou fundamental ou ônibus de traslado em uma instituição de ensino superior não cobram tarifas.","Outros tipos, como os autocarros da escola primária ou secundária ou de ida e volta num campus pós-secundário não cobram bilhete.",None
...,...,...,...,...,...,...
3103,3104,Gold Collection,n/a,"Isso significa que a preposição é o termo que relaciona substantivo a substantivo, verbo a substantivo, substantivo a verbo, adjetivo a substantivo, advérbio a substantivo, etc.\n","Isso significa que a preposição é o termo que relaciona substantivo a substantivo, verbo a substantivo, substantivo a verbo, adjetivo a substantivo, advérbio a substantivo, etc.\n","Isto significa que uma preposição é o termo que relaciona um substantivo com um substantivo, um verbo com um substantivo, um substantivo com um verbo, um adjetivo com um substantivo, um advérbio com um substantivo, etc.\n"
3104,3105,Gold Collection,n/a,Foi então que a vestimenta mais feminina que se conhece começou a ganhar forma: o espartilho.\n,Foi então que a peça de vestuário mais feminina que se conhece começou a ganhar forma: o espartilho.\n,Foi nessa altura que começou a ganhar forma a peça de vestuário mais feminina que se conhece: o espartilho.\n
3105,3106,Gold Collection,n/a,Um de seus professores foi Martin Wegelius.\n,Um dos seus professores foi Martin Wegelius.\n,Um dos seus professores foi Martin Wegelius.\n
3106,3107,Gold Collection,n/a,"Nessa época, iniciou uma verdadeira polêmica com o escritor democrata Björnstjerne Björnson, através de correspondência.\n","Nessa época, iniciou-se uma verdadeira polémica com o escritor democrata Björnstjerne Björnson, através de correspondência.\n","Nesta época, iniciou uma verdadeira polémica com o escritor democrata Björnstjerne Björnson, através de correspondência.\n"


In [14]:
def sample_test_df(DB_PATH, dataset, n=5):
    """Return a random sample of `n` rows from the DataFrame."""
    con = duckdb.connect(DB_PATH)
    df = con.execute(f"SELECT * FROM test_data WHERE dataset = '{dataset}' LIMIT {n}").fetchdf()
    con.close()
    return df

def sample_train_df(DB_PATH, dataset, n=5):
    """Return a random sample of `n` rows from the DataFrame."""
    con = duckdb.connect(DB_PATH)
    df = con.execute(f"SELECT * FROM train_data WHERE dataset = '{dataset}' LIMIT {n}").fetchdf()
    con.close()
    return df

In [13]:
sample_test_df(DB_PATH, "FRMT")

,test_id,dataset,bucket,text_pt_br,text_pt_pt
0,1,FRMT,lexical,"Um ônibus (contração de ômnibus, com as variações multibus, motorbus, autobus etc.) é um veículo rodoviário projetado para transportar muitos passageiros.",Um autocarro é um veículo rodoviário concebido para transportar muitos passageiros.
1,2,FRMT,lexical,Ônibus podem ter capacidade para até 300 passageiros.,Os autocarros têm capacidade para transportar no máximo 300 passageiros.
2,3,FRMT,lexical,"O tipo de ônibus mais comum é o ônibus rígido de um andar, com cargas maiores transportadas por ônibus de dois andares e articulados, e cargas menores transportadas por ônibus médios e micro-ônibus, enquanto ônibus executivos são usados para serviços de distâncias maiores.","Os tipos de autocarros mais comuns são os rígidos de um andar, sendo as cargas maiores nos de dois andares e os autocarros articulados. As cargas menores transportadas por autocarros médios e miniautocarros e enquanto as camionetas de passageiros são utilizadas para serviços de maior distância."
3,4,FRMT,lexical,"Muitos tipos de ônibus, como os de transporte urbano e os intermunicipais, cobram uma tarifa.","Muitos tipos de autocarros, como os de trânsito urbano e intercidades, cobram bilhete."
4,5,FRMT,lexical,"Outros tipos, como ônibus escolares do ensino médio ou fundamental ou ônibus de traslado em uma instituição de ensino superior não cobram tarifas.","Outros tipos, como os autocarros da escola primária ou secundária ou de ida e volta num campus pós-secundário não cobram bilhete."


In [5]:
con.close()

vale a pena manter frases iguais nos dados?

**//TRAIN DATASETS**

In [4]:
con = duckdb.connect(DB_PATH)


In [5]:
con.execute("SELECT * FROM ptbrvarid WHERE dataset = 'train'").fetchdf()

,dataset,split,label,text_pt_br,text_pt_pt


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# con = duckdb.connect(DB_PATH)
# con.execute("""CREATE TABLE IF NOT EXISTS ptbrvarid (
#     dataset     TEXT,              -- always 'PtBrVarId'
#     split       TEXT,              -- 'train' | 'valid' | 'test'
#     label       TEXT,              -- 'pt-BR' | 'pt-PT'
#     text_pt_br  TEXT,
#     text_pt_pt  TEXT
# );""")

In [6]:
from datasets import load_dataset
import duckdb, pandas as pd, tqdm

DB_PATH = "../data/duckdb/subs.duckdb"
domains = ["journalistic", "legal", "literature",
           "politics", "social_media", "web"]

with duckdb.connect(DB_PATH) as con:
    # ensure the storage table exists once
    con.execute("""
        CREATE TABLE IF NOT EXISTS ptbrvarid (
            dataset     TEXT,
            split       TEXT,
            label       TEXT,
            text_pt_br  TEXT,
            text_pt_pt  TEXT
        );
    """)

    BATCH = 50_000
    buffer = []

    for cfg in domains:
        ds_cfg = load_dataset("liaad/PtBrVId", cfg, streaming=True)  # stream!

        for split in ("train", "valid"):           # leave 'test' for later
            for ex in tqdm.tqdm(ds_cfg[split],
                                desc=f"{cfg}-{split}",
                                unit="rows"):
                if ex["label"] == 1:               # 1 = BR
                    buffer.append(("PtBrVarId", split, "pt-BR",
                                    ex["text"], None))
                else:                              # 0 = PT
                    buffer.append(("PtBrVarId", split, "pt-PT",
                                    None, ex["text"]))

                if len(buffer) >= BATCH:
                    con.executemany(
                        "INSERT INTO ptbrvarid VALUES (?, ?, ?, ?, ?)",
                        buffer)
                    buffer.clear()

    # flush remainder
    if buffer:
        con.executemany(
            "INSERT INTO ptbrvarid VALUES (?, ?, ?, ?, ?)",
            buffer)


journalistic-train: 1776290rows [4:56:50, 99.73rows/s]   
journalistic-valid: 1000rows [00:03, 275.65rows/s]
legal-train: 1122709rows [2:53:44, 107.70rows/s]  


KeyboardInterrupt: 

In [ ]:
rows_test = []
for cfg in domains:
    for ex in load_dataset("liaad/PtBrVId", cfg, split="test"):
        if ex["label"] == 0:
            rows_test.append(("PtBrVarId", "n/a", ex["text"], None))
        else:
            rows_test.append(("PtBrVarId", "n/a", None, ex["text"]))

with duckdb.connect(DB_PATH) as con:
    con.executemany(
        "INSERT INTO test_data (dataset, bucket, text_pt_br, text_pt_pt) "
        "VALUES (?, ?, ?, ?)",
        rows_test)


In [ ]:
con.execute("""
DROP VIEW IF EXISTS train_data;

CREATE VIEW train_data AS

-- OpenSubtitles: always used for translation
SELECT
       'OpenSubs'  AS dataset,
       sent_pt_br  AS text_pt_br,
       sent_pt_pt  AS text_pt_pt,
       NULL        AS label                           -- no class label
FROM   opus_moses

UNION ALL

-- PtBrVarId: keep just train & valid rows
SELECT
       'PtBrVarId' AS dataset,
       text_pt_br,
       text_pt_pt,
       label                                        -- 'pt-BR' / 'pt-PT'
FROM   ptbrvarid
WHERE  split IN ('train','valid');


""")

In [12]:
con.execute("SELECT * FROM train_data WHERE split = 'valid' LIMIT 5").fetchdf()

,dataset,split,label,text_pt_br,text_pt_pt
0,PtBrVarId,valid,pt-PT,None,None
1,PtBrVarId,valid,pt-PT,None,None
2,PtBrVarId,valid,pt-PT,None,None
3,PtBrVarId,valid,pt-PT,None,None
4,PtBrVarId,valid,pt-PT,None,None


In [17]:
sample_train_df(DB_PATH, "OpenSubs")

,dataset,split,label,text_pt_br,text_pt_pt
0,OpenSubs,train,None,"o diretor mueller acaba de nomear nos um número de caso principal, investigações foi oficialmente dublado amerithrax, quemosenviou cartas receberam antraz de um laboratório americano, nós não estaríamos aqui se nós não tinha provas nos conduzindo de volta para usamriid,","Em episódios anteriores... O diretor Mueller atribuiu-nos um caso importante. Oficialmente, a investigação chama-se Amerithrax."
1,OpenSubs,train,None,"bruce ivins é o nosso melhor homem com antraz, então eu quero falar com ele, eles estão nos tratando como se fôssemos o inimigo, nós somos os heróis, a entrada tem que ser através do complexo de seu herói,",- Bruce Ivins é o perito em antraz. - Então quero falar com ele. Tratam-nos como o inimigo e somos os heróis.
2,OpenSubs,train,None,"eu conheço a maioria dos americanos não estão prestando atenção, mas vocês tiveram sinais que o 11 de setembro iria acontecer, eagoravocênão consegueentender quem enviou essas cartas, então, em vez disso, você está assediando patriotas que trabalham duro como eu,","Temos de entrar através do complexo de herói dele. A maioria dos americanos não presta atenção, mas vocês tiveram sinais de que o 11 de Setembro ia acontecer. E agora não descobrem quem enviou as cartas."
3,OpenSubs,train,None,"bruce,vocêé considerando fazer fisicamente mal a alguém?","E, em vez disso, incomodam patriotas trabalhadores como eu. Bruce, está a pensar em magoar alguém?"
4,OpenSubs,train,None,"você fez isso, dr, ivins?","Fez isto, Dr. Ivins?"


In [ ]:
df_test = con.execute("SELECT * FROM train_data WHERE split = 'train'").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
df_test

,dataset,split,label,text_pt_br,text_pt_pt
